In [1]:
from pathlib import Path
import nibabel as nib
import numpy as np

folder = Path(r"Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0x7axQ0Q1\manual_segmentations\nii")

empty_files = []

for nii_path in folder.glob("*.nii*"):
    try:
        img = nib.load(nii_path)
        data = img.get_fdata()
        if np.all(data == 0) or data.size == 0:
            empty_files.append(nii_path.name)
    except Exception as e:
        print(f"Error loading {nii_path.name}: {e}")

if empty_files:
    print("\nEmpty NIfTI files:")
    for f in empty_files:
        print(" -", f)
else:
    print("No empty NIfTI files found.")



Empty NIfTI files:
 - I1.nii.gz
 - Isocenter_1.nii.gz
 - Isocenter_2.nii.gz
 - Isocenter_3.nii.gz
 - Isocenter_4.nii.gz
 - Isocenter_5.nii.gz
 - Isocenter_6.nii.gz
 - Isocenter_7.nii.gz
 - M1.nii.gz


In [2]:
import nibabel as nib
import numpy as np
from pathlib import Path

right_path = Path(r"Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0x7axQ0Q1\comparison\Mandible_R.nii.gz")
left_path = Path(r"Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0x7axQ0Q1\comparison\Mandible_L.nii.gz")
out_path = Path(r"Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0x7axQ0Q1\comparison\mandible_manual.nii.gz")

right_img = nib.load(right_path)
left_img = nib.load(left_path)

right_data = right_img.get_fdata()
left_data = left_img.get_fdata()

combined_data = np.logical_or(right_data, left_data).astype(np.uint8)

combined_img = nib.Nifti1Image(combined_data, affine=right_img.affine, header=right_img.header)

nib.save(combined_img, out_path)

print(f"Combined mandible saved to: {out_path}")

Combined mandible saved to: Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0x7axQ0Q1\comparison\mandible_manual.nii.gz


In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import nibabel as nib
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output

ct_img = nib.load(r"Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0x7axQ0Q1\CT.nii.gz")
X = ct_img.get_fdata()
X = (X - np.min(X)) / (np.max(X) - np.min(X))  # Normalize to [0, 1] (binary)

seg_paths = {
    "mandible_manual": r"Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0x7axQ0Q1\comparison\mandible_manual.nii.gz",
    #"mandible_totalseg": r"Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0x1x1Hx7\Total_Segmentator\mandible.nii.gz",
    #"Mandible_R": r"Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0x1x1Hx7\manual_segmentations\nii\Mandible_R.nii.gz",
    #"Mandible_R_(1)": r"Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0x1x1Hx7\manual_segmentations\nii\Mandible_R_(1).nii.gz",
    "Dentition": r"Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0x7axQ0Q1\comparison\Dentition.nii.gz",
    #"teeth_upper": r"Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0x1x1Hx7\Total_Segmentator\teeth_upper.nii.gz",
    #"teeth_lower": r"Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0x1x1Hx7\Total_Segmentator\teeth_lower.nii.gz",
}

segments = {name: nib.load(path).get_fdata() for name, path in seg_paths.items()}
spacing = ct_img.header.get_zooms()

overlay_colors = {
    "mandible_manual": (0.2, 0.4, 1.0, 0.25),          # blueish
    #"mandible_totalseg": (1.0, 0.6, 0.0, 0.35),      # orange
    #"Mandible_R": (0.2, 1.0, 0.2, 0.35), # green
    "Dentition": (0.6, 0.2, 1.0, 0.35), # purple
    #"M1": (0.9, 0.9, 0.8, 0.25),         # bone
   # "teeth_upper": (1.0, 0.2, 0.2, 0.35),   # red
   # "teeth_lower": (1.0, 0.6, 0.4, 0.35),   # light red/orange
}

def show_slice(slice_index, plane, alpha=0.35, patient_id="UIDQQ0x1x1Hx7"):
    plt.figure(figsize=(6, 6))

    if plane == 'Axial':
        img_slice = np.rot90(X[:, :, slice_index])
        aspect = spacing[1] / spacing[0]
        seg_slices = {n: np.rot90(S[:, :, slice_index]) for n, S in segments.items()}

    elif plane == 'Coronal':
        img_slice = np.flipud(np.rot90(X[:, slice_index, :]))
        aspect = spacing[2] / spacing[0]
        seg_slices = {n: np.flipud(np.rot90(S[:, slice_index, :])) for n, S in segments.items()}

    elif plane == 'Sagittal':
        img_slice = np.flipud(np.rot90(X[slice_index, :, :]))
        aspect = spacing[2] / spacing[1]
        seg_slices = {n: np.flipud(np.rot90(S[slice_index, :, :])) for n, S in segments.items()}

    plt.imshow(img_slice, cmap='gray', origin='lower', aspect=aspect)

    for name, seg_slice in seg_slices.items():
        mask = seg_slice > 0.5
        if np.any(mask):
            rgba = list(overlay_colors[name])
            rgba[3] = alpha  
            overlay = np.zeros((*mask.shape, 4))
            overlay[mask] = rgba
            plt.imshow(overlay, origin='lower', aspect=aspect)

    plt.title(f"{plane} slice {slice_index}")
    plt.axis("off")


    legend_handles = [
        plt.Line2D([0], [0], color=overlay_colors[n][:3], lw=4, label=n)
        for n in overlay_colors
    ]
    leg=plt.legend(handles=legend_handles, loc='upper right', fontsize=8, frameon=False, labelcolor='white', title="Segments")
    leg.get_title().set_fontsize(10)
    leg.get_title().set_color('white')
    clear_output(wait=True)
    display(plt.gcf())
    plt.close()


plane_dropdown = widgets.Dropdown(
    options=['Axial', 'Coronal', 'Sagittal'],
    value='Axial',
    description='Plane:'
)

slice_slider = widgets.IntSlider(
    value=X.shape[2] // 2,
    min=0,
    max=X.shape[2] - 1,
    step=1,
    description='Slice:',
    continuous_update=True,
    layout=widgets.Layout(width='80%')
)

alpha_slider = widgets.FloatSlider(
    value=0.35,
    min=0.0,
    max=1.0,
    step=0.05,
    description='Opacity:',
    continuous_update=True,
    layout=widgets.Layout(width='60%')
)


def update_slider_range(*args):
    plane = plane_dropdown.value
    if plane == 'Axial':
        slice_slider.max = X.shape[2] - 1
        slice_slider.value = X.shape[2] // 2
    elif plane == 'Coronal':
        slice_slider.max = X.shape[1] - 1
        slice_slider.value = X.shape[1] // 2
    elif plane == 'Sagittal':
        slice_slider.max = X.shape[0] - 1
        slice_slider.value = X.shape[0] // 2


plane_dropdown.observe(update_slider_range, names='value')
update_slider_range()

widgets.interact(show_slice, slice_index=slice_slider, plane=plane_dropdown, alpha=alpha_slider)


interactive(children=(IntSlider(value=120, description='Slice:', layout=Layout(width='80%'), max=239), Dropdow…

<function __main__.show_slice(slice_index, plane, alpha=0.35, patient_id='UIDQQ0x1x1Hx7')>

In [ ]:
#to view on subplots
%matplotlib inline

import matplotlib.pyplot as plt
import nibabel as nib
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output

ct_img = nib.load(r"Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0x1x1Hx7\CT.nii.gz")
X = ct_img.get_fdata()
X = (X - np.min(X)) / (np.max(X) - np.min(X))  # Normalize to [0, 1]
spacing = ct_img.header.get_zooms()

seg_paths = {
    "mandible_manual": r"Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0x1x1Hx7\comparison\mandible_manual.nii.gz",
    "mandible_totalseg": r"Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0x1x1Hx7\comparison\mandible_totalseg.nii.gz",
    "Dentition": r"Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0x1x1Hx7\comparison\Dentition.nii.gz",
    "teeth": r"Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0x1x1Hx7\comparison\teeth.nii.gz",
}
segments = {name: nib.load(path).get_fdata() for name, path in seg_paths.items()}

overlay_colors = {
    "mandible_manual": (0.2, 0.4, 1.0, 0.25),   # blue
    "mandible_totalseg": (0.2, 0.4, 1.0, 0.25),
    "Dentition": (0.6, 0.2, 1.0, 0.35),         # purple
    "teeth": (0.6, 0.2, 1.0, 0.35),
}

def show_slice(slice_index, plane, alpha=0.35):
    fig, axes = plt.subplots(1, 2, figsize=(12, 6))
    plt.tight_layout()

    if plane == 'Axial':
        img_slice = np.rot90(X[:, :, slice_index])
        aspect = spacing[1] / spacing[0]
        seg_slices = {n: np.rot90(S[:, :, slice_index]) for n, S in segments.items()}

    elif plane == 'Coronal':
        img_slice = np.flipud(np.rot90(X[:, slice_index, :]))
        aspect = spacing[2] / spacing[0]
        seg_slices = {n: np.flipud(np.rot90(S[:, slice_index, :])) for n, S in segments.items()}

    elif plane == 'Sagittal':
        img_slice = np.flipud(np.rot90(X[slice_index, :, :]))
        aspect = spacing[2] / spacing[1]
        seg_slices = {n: np.flipud(np.rot90(S[slice_index, :, :])) for n, S in segments.items()}

    # left subplot 
    ax_left = axes[0]
    ax_left.imshow(img_slice, cmap='gray', origin='lower', aspect=aspect)

    for name in ["mandible_totalseg", "teeth"]:
        if name in seg_slices:
            mask = seg_slices[name] > 0.5
            if np.any(mask):
                rgba = list(overlay_colors[name])
                rgba[3] = alpha
                overlay = np.zeros((*mask.shape, 4))
                overlay[mask] = rgba
                ax_left.imshow(overlay, origin='lower', aspect=aspect)

    ax_left.set_title(f"{plane} slice {slice_index} — TotalSegmentator")
    ax_left.axis("off")

    legend_handles_left = [
        plt.Line2D([0], [0], color=overlay_colors[n][:3], lw=4, label=n)
        for n in ["mandible_totalseg", "teeth"]
    ]
    leg_left = ax_left.legend(
        handles=legend_handles_left,
        loc='lower right',
        fontsize=8,
        frameon=True,
        facecolor='black',
        edgecolor='white',
        labelcolor='white',
        title="Segments",
        title_fontsize=9
    )

    plt.setp(leg_left.get_texts(), color='white')
    leg_left.get_title().set_color('white')

    # right subplot
    ax_right = axes[1]
    ax_right.imshow(img_slice, cmap='gray', origin='lower', aspect=aspect)

    for name in ["Dentition", "mandible_manual"]:
        if name in seg_slices:
            mask = seg_slices[name] > 0.5
            if np.any(mask):
                rgba = list(overlay_colors[name])
                rgba[3] = alpha
                overlay = np.zeros((*mask.shape, 4))
                overlay[mask] = rgba
                ax_right.imshow(overlay, origin='lower', aspect=aspect)

    ax_right.set_title(f"{plane} slice {slice_index} — Manual Segmentations")
    ax_right.axis("off")

    legend_handles_right = [
        plt.Line2D([0], [0], color=overlay_colors[n][:3], lw=4, label=n)
        for n in ["Dentition", "mandible_manual"]
    ]
    leg_right = ax_right.legend(
        handles=legend_handles_right,
        loc='lower right',
        fontsize=8,
        frameon=True,
        facecolor='black',
        edgecolor='white',
        labelcolor='white',
        title="Segments",
        title_fontsize=9
    )

    plt.setp(leg_right.get_texts(), color='white')
    leg_right.get_title().set_color('white')

    plt.subplots_adjust(wspace=0.02)
    clear_output(wait=True)
    display(fig)
    plt.close(fig)


plane_dropdown = widgets.Dropdown(
    options=['Axial', 'Coronal', 'Sagittal'],
    value='Axial',
    description='Plane:'
)

slice_slider = widgets.IntSlider(
    value=X.shape[2] // 2,
    min=0,
    max=X.shape[2] - 1,
    step=1,
    description='Slice:',
    continuous_update=True,
    layout=widgets.Layout(width='80%')
)

alpha_slider = widgets.FloatSlider(
    value=0.35,
    min=0.0,
    max=1.0,
    step=0.05,
    description='Opacity:',
    continuous_update=True,
    layout=widgets.Layout(width='60%')
)

def update_slider_range(*args):
    plane = plane_dropdown.value
    if plane == 'Axial':
        slice_slider.max = X.shape[2] - 1
        slice_slider.value = X.shape[2] // 2
    elif plane == 'Coronal':
        slice_slider.max = X.shape[1] - 1
        slice_slider.value = X.shape[1] // 2
    elif plane == 'Sagittal':
        slice_slider.max = X.shape[0] - 1
        slice_slider.value = X.shape[0] // 2

plane_dropdown.observe(update_slider_range, names='value')
update_slider_range()

widgets.interact(show_slice, slice_index=slice_slider, plane=plane_dropdown, alpha=alpha_slider)


interactive(children=(IntSlider(value=122, description='Slice:', layout=Layout(width='80%'), max=244), Dropdow…

<function __main__.show_slice(slice_index, plane, alpha=0.35)>